In this project we will be applying different types of forecasting techniques on the given cpu data and analyise which one of them provides better accuracy and why.

We will be using folowing forecasting techniques:

1) Simple Moving Average

2) Weighted Moving average

3) Exponential Smoothning

4) Exponential Smoothning w/ trend

5) Moving weighted average EST

6) Linear Regression

7) Non-linear Regression (2nd order)

# Pre-Processing

Importing all the required libraries

In [1]:
import pandas as pd
import numpy as np
import cPickle
import time
import pyprind
import plotly.plotly as py
import plotly.graph_objs as go
import scipy

Linux src files

In [2]:
src_file = "/home/narsi/Documents/CSEE5690_data/VM-CPU-Stats-1-Day.csv"
store_file = file('/home/narsi/Documents/CSEE5690_data/cpu_analysis/cpu_analytics_'+time.strftime("%Y%m%d-%H%M%S")+'.pkl', 'wb')

Windows src files

In [2]:
src_file = "D:\\course_related_data\\CSEE5690\\VM-CPU-Stats-1-Day.csv"
store_file = file('D:\\course_related_data\\CSEE5690\\cpu_analytics_'+time.strftime("%Y%m%d-%H%M%S")+'.pkl', 'wb')

Read Data from csv file

In [3]:
cpu_data = pd.read_csv(src_file).iloc[::-1]

In [4]:
cpu_freq = np.array(cpu_data['Value'],dtype = np.float32)
E_id_list = cpu_data.Entity.unique()
E_select = [25,60,110,225,350]

In [14]:
for i in E_select:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[i]].index.tolist(), dtype=np.int32))-1
    x = len(id_list)
    print(E_id_list[i] , ' Mean' , np.sum(cpu_freq[id_list])/x , ' Var' , np.var(cpu_freq[id_list]))

('kc-itsm-pweb01', ' Mean', 884.72318339100343, ' Var', 824479.56)
('Wulfy-Compute03', ' Mean', 6.1130136986301373, ' Var', 0.10024159)
('kc-pd-sql01', ' Mean', 281.01027397260276, ' Var', 447125.25)
('kc-issr-nmap1', ' Mean', 34.553264604810998, ' Var', 1414.6252)
('KC-ISFS-DCIM01', ' Mean', 28.455172413793104, ' Var', 14476.157)


In [17]:
# [3, 5, 9, 11, 20]
trace_w3 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[0]].index.tolist(), dtype=np.int32))-1], marker={'size': "2"}, 
                    mode="lines", name='kc-itsm-pweb01 Mean:884.72 Var:824479.56')
trace_w5 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[1]].index.tolist(), dtype=np.int32))-1], marker={'size': "5"}, 
                    mode="lines", name='Wulfy-Compute03 Mean:6.11 Var:0.10')
trace_w9 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[2]].index.tolist(), dtype=np.int32))-1], marker={'size': "9"}, 
                    mode="lines", name='kc-pd-sql01 Mean:281.01 Var:447125.25')
trace_w11 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[3]].index.tolist(), dtype=np.int32))-1], marker={'size': "12"}, 
                    mode="lines", name='kc-issr-nmap1 Mean:34.55 Var:1414.6252')
trace_w20 = go.Scatter(y=cpu_freq[(len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id_list[4]].index.tolist(), dtype=np.int32))-1], marker={'size': "15"}, 
                    mode="lines", name='KC-ISFS-DCIM01 Mean:28.46 Var:14476.157')


data=go.Data([trace_w3,trace_w5,trace_w9,trace_w11,trace_w20])
layout=go.Layout(title="Randomly Select Virtual Machines")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

In [11]:
cpu_freq[id_list]

array([], dtype=float32)

In [5]:
cpu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146004 entries, 146003 to 0
Data columns (total 9 columns):
Value           146004 non-null int64
Timestamp       146004 non-null object
MetricId        146004 non-null object
Unit            146004 non-null object
Description     146004 non-null object
Entity          146004 non-null object
EntityId        146004 non-null object
IntervalSecs    146004 non-null int64
Instance        0 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 10.0+ MB


# 1) Moving Average

In [18]:
MA = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(len(E_select)*5, stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list) - window)
        counter = 0
        for i in id_list[window:]:
            X = cpu_freq[i - window:i]
            x[counter] = np.sum(X) * 1.0 / window
            counter += 1

        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        #MA.append({'Window': window, 'Actual_Data': cpu_freq[id_list[window:]], 'Predicted': x, 'TS_Score': TS}, ignore_index=True)
        MA.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()


0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [20]:
MA.to_csv('D:\\course_related_data\\CSEE5690\\ma.csv')

In [8]:
trace_max4 = go.Scatter( x = E_id_list[E_select],y=[4] * len(E_select), marker={'size': "5"}, 
                    mode="lines", name='Max Limit')
trace_min4 = go.Scatter( x = E_id_list[E_select],y=[-4] * len(E_select), marker={'size': "5"}, 
                    mode="lines", name='Min Limit')

In [111]:
# [3, 5, 9, 11, 20]
trace_w3 = go.Scatter( x = E_id_list[E_select],y=MA[MA['Window'] == 3].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 3 array')
trace_w5 = go.Scatter( x = E_id_list[E_select],y=MA[MA['Window'] == 5].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 5 array')
trace_w9 = go.Scatter( x = E_id_list[E_select],y=MA[MA['Window'] == 9].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 9 array')
trace_w11 = go.Scatter( x = E_id_list[E_select],y=MA[MA['Window'] == 11].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 11 array')
trace_w20 = go.Scatter( x = E_id_list[E_select],y=MA[MA['Window'] == 20].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 20 array')


data=go.Data([trace_w3,trace_w5,trace_w9,trace_w11,trace_w20,trace_max4,trace_min4])
layout=go.Layout(title="Moving Average")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

# 2) Weighted Moving Average

In [23]:
WMA = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(len(E_select)*5, stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list)-window)
        counter = 0
        weight_ = np.arange(window,dtype = np.float32)+1
        weight_ = weight_/np.sum(weight_)
        
        for i in id_list[window:]:
            x[counter] = np.sum(cpu_freq[i-window:i] * weight_)
            counter += 1
        
        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        WMA.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()


0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [25]:
WMA.to_csv('D:\\course_related_data\\CSEE5690\\ma.csv')

In [110]:
# [3, 5, 9, 11, 20]
trace_wma3 = go.Scatter( x = E_id_list[E_select],y=WMA[WMA['Window'] == 3].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 3 array')
trace_wma5 = go.Scatter( x = E_id_list[E_select],y=WMA[WMA['Window'] == 5].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 5 array')
trace_wma9 = go.Scatter( x = E_id_list[E_select],y=WMA[WMA['Window'] == 9].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 9 array')
trace_wma11 = go.Scatter( x = E_id_list[E_select],y=WMA[WMA['Window'] == 11].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 11 array')
trace_wma20 = go.Scatter( x = E_id_list[E_select],y=WMA[WMA['Window'] == 20].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 20 array')

data=go.Data([trace_wma3,trace_wma5,trace_wma9,trace_wma11,trace_wma20,trace_max4,trace_min4])
layout=go.Layout(title="Weighted Moving Average")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

# 3) Exponential Smoothing

In [29]:
ES = pd.DataFrame(columns=['Subject','Alpha','TS_Score','MAD','A','F'])

alpha = [0.2,0.4,0.6,0.8]
dataframe_counter = 0
bar = pyprind.ProgBar(len(alpha)*len(E_select), stream=1)
for a in alpha:
    for E_id in E_id_list[E_select]:
        id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype = np.int32))-1
        x = np.empty(len(id_list))
        x[0] = cpu_freq[id_list[0]]
        counter = 1
        for i in id_list[1:]:
            x[counter] = x[counter-1] + (a) * (cpu_freq[i-1] - x[counter-1])
            counter += 1
        
        TS = np.sum(cpu_freq[id_list]-x)/(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x))
        ES.loc[dataframe_counter] = [E_id,a,TS,(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x)),cpu_freq[id_list],x]
        dataframe_counter +=1
        bar.update()

0%                100%
[####################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [30]:
ES.to_csv('D:\\course_related_data\\CSEE5690\\ma.csv')

In [109]:
trace_a2 = go.Scatter( x = E_id_list[E_select],y=ES[ES['Alpha'] == 0.2].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.2')
trace_a4 = go.Scatter( x = E_id_list[E_select],y=ES[ES['Alpha'] == 0.4].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.4')
trace_a6 = go.Scatter( x = E_id_list[E_select],y=ES[ES['Alpha'] == 0.6].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.6')
trace_a8 = go.Scatter( x = E_id_list[E_select],y=ES[ES['Alpha'] == 0.8].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.8')

data=go.Data([trace_a2,trace_a4,trace_a6,trace_a8,trace_max4,trace_min4])
layout=go.Layout(title="Exponential Smoothing")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

# 4) Exponential Smoothing with trend

In [31]:
EST = pd.DataFrame(columns=['Subject','Alpha','TS_Score','MAD','A','F'])
d = 0.5
alpha = [0.2,0.4,0.6,0.8]
dataframe_counter = 0
bar = pyprind.ProgBar(len(alpha)*len(E_select), stream=1)
for a in alpha:
    for E_id in E_id_list[E_select]:
        id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype = np.int32))-1
        x = np.empty(len(id_list))
        x[0] = cpu_freq[id_list[0]]
        T = 0.0
        F = x[0]
        counter = 1
        for i in id_list[1:]:
            F = x[counter - 1] + a * (cpu_freq[i-1] - x[counter-1])
            T = T + d * (F - x[counter-1])
            x[counter] = F + T
            counter += 1

        TS = np.sum(cpu_freq[id_list]-x)/(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x))
        EST.loc[dataframe_counter] = [E_id,a,TS,(np.sum(np.absolute(cpu_freq[id_list]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()


0%                100%
[####################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [32]:
EST.to_csv('D:\\course_related_data\\CSEE5690\\ma.csv')

In [117]:
trace_ad2 = go.Scatter( x = E_id_list[E_select],y=EST[EST['Alpha'] == 0.2].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.2')
trace_ad4 = go.Scatter( x = E_id_list[E_select],y=EST[EST['Alpha'] == 0.4].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.4')
trace_ad6 = go.Scatter( x = E_id_list[E_select],y=EST[EST['Alpha'] == 0.6].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.6')
trace_ad8 = go.Scatter( x = E_id_list[E_select],y=EST[EST['Alpha'] == 0.8].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.8')

data=go.Data([trace_ad2,trace_ad4,trace_ad6,trace_ad8,trace_max4,trace_min4])
layout=go.Layout(title="Exponential Smoothing with trend")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

# 5) Windowed Exponential Smoothing with trend V1

In [64]:
WEST = pd.DataFrame(columns=['Subject','Alpha','Window','TS_Score','A','F'])
d = 0.5
alpha = [0.2,0.4,0.6,0.8]
bar = pyprind.ProgBar(5*4*len(E_select), stream=1)
dataframe_counter = 0
for a in alpha:
    for E_id in E_id_list[E_select]:
        id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype = np.int32))-1
        for window in [3, 5, 9, 11, 20]:
            x = np.empty(len(id_list))
            weight_ = np.arange(window,dtype = np.float32)+1
            weight_ = weight_/np.sum(weight_)
            x[0:window] = cpu_freq[id_list[0:window]]
            T = 0.0
            F = x[window-1]
            counter = window
            for i in id_list[window:]:
                F = x[counter - 1] + a * (np.sum((cpu_freq[i-window:i] - x[counter-window:counter])*weight_))
                T = T + d * (F - x[counter-1])
                x[counter] = F + T
                if x[counter] <= 0:
                    x[counter] = np.sum(cpu_freq[i-window:i]*weight_)
                counter += 1

            TS = np.sum(cpu_freq[id_list[window:]]-x[window:])/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x[window:]))/len(x))
            WEST.loc[dataframe_counter] = [E_id,a,window,TS,cpu_freq[id_list[window:]],x[window:]]
            dataframe_counter += 1
            bar.update()


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [65]:
trace_wad2 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.2) & (WEST['Window'] == 3)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.2')
trace_wad4 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.4) & (WEST['Window'] == 3)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.4')
trace_wad6 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.6) & (WEST['Window'] == 3)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.6')
trace_wad8 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.8) & (WEST['Window'] == 3)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.8')

data=go.Data([trace_wad2,trace_wad4,trace_wad6,trace_wad8,trace_max4,trace_min4])
layout=go.Layout(title="weighted window Exponential Smoothing with trend w/ window size of 3 and delat of 0.5")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

PlotlyError: Request throttled. You've created/updated more charts than your allowed limit of 30/hour. You may either wait one hour or upgrade your account. Visit https://plot.ly/settings/subscription/ to upgrade.

In [59]:
trace_wad2 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.2) & (WEST['Window'] == 5)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.2')
trace_wad4 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.4) & (WEST['Window'] == 5)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.4')
trace_wad6 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.6) & (WEST['Window'] == 5)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.6')
trace_wad8 = go.Scatter( x = E_id_list[E_select],y=WEST[(WEST['Alpha'] == 0.8) & (WEST['Window'] == 5)].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Alpha 0.8')

data=go.Data([trace_wad2,trace_wad4,trace_wad6,trace_wad8,trace_max4,trace_min4])
layout=go.Layout(title="weighted window Exponential Smoothing with trend w/ window size of 3 and delat of 0.5")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

# 6) Piecewise Linear

In [38]:
PL = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(5*len(E_select), stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list)-window)
        t = np.arange(window)
        weight_ = np.arange(window,dtype = np.float32)+1
        weight_ = weight_/np.sum(weight_)
        counter = 0
        for i in id_list[window:]:
            (ar,br) = scipy.polyfit(t,cpu_freq[i-window:i],1)
            x[counter] = scipy.polyval([ar,br],window)
            counter += 1
            
        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        PL.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()


0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [39]:
PL.to_csv('D:\\course_related_data\\CSEE5690\\ma.csv')

In [9]:
# [3, 5, 9, 11, 20]
trace_lw3 = go.Scatter( x = E_id_list[E_select],y=PL[PL['Window'] == 3].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 3 array')
trace_lw5 = go.Scatter( x = E_id_list[E_select],y=PL[PL['Window'] == 5].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 5 array')
trace_lw9 = go.Scatter( x = E_id_list[E_select],y=PL[PL['Window'] == 9].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 9 array')
trace_lw11 = go.Scatter( x = E_id_list[E_select],y=PL[PL['Window'] == 11].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 11 array')
trace_lw20 = go.Scatter( x = E_id_list[E_select],y=PL[PL['Window'] == 20].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 20 array')

data=go.Data([trace_lw3,trace_lw5,trace_lw9,trace_lw11,trace_lw20,trace_max4,trace_min4])
layout=go.Layout(title="Piecewise Linear")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

# 7) Piecewise Polynomial

In [40]:
PL2 = pd.DataFrame(columns=['Subject','Window','TS_Score','MAD','A','F'])
bar = pyprind.ProgBar(5*len(E_select), stream=1)
dataframe_counter = 0
for E_id in E_id_list[E_select]:
    id_list = (len(cpu_data) - np.array(cpu_data[cpu_data['Entity'] == E_id].index.tolist(), dtype=np.int32))-1
    for window in [3, 5, 9, 11, 20]:
        x = np.empty(len(id_list)-window)
        t = np.arange(window)
        weight_ = np.arange(window,dtype = np.float32)+1
        weight_ = weight_/np.sum(weight_)
        counter = 0
        for i in id_list[window:]:
            z = np.polyfit(t,cpu_freq[i-window:i],2)
            p = np.poly1d(z)
            x[counter] = p(window)
            counter += 1
            
        TS = np.sum(cpu_freq[id_list[window:]]-x)/(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x))
        PL2.loc[dataframe_counter] = [E_id,window,TS,(np.sum(np.absolute(cpu_freq[id_list[window:]]-x))/len(x)),cpu_freq[id_list[window:]],x]
        dataframe_counter += 1
        bar.update()


0%                     100%
[#########################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [41]:
PL2.to_csv('D:\\course_related_data\\CSEE5690\\ma.csv')

In [37]:
# [3, 5, 9, 11, 20]
trace_pw3 = go.Scatter( x = E_id_list[E_select],y=PL2[PL2['Window'] == 3].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 3 array')
trace_pw5 = go.Scatter( x = E_id_list[E_select],y=PL2[PL2['Window'] == 5].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 5 array')
trace_pw9 = go.Scatter( x = E_id_list[E_select],y=PL2[PL2['Window'] == 9].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 9 array')
trace_pw11 = go.Scatter( x = E_id_list[E_select],y=PL2[PL2['Window'] == 11].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 11 array')
trace_pw20 = go.Scatter( x = E_id_list[E_select],y=PL2[PL2['Window'] == 20].TS_Score.tolist(), marker={'size': "10"}, 
                    mode="markers", name='Weight 20 array')

data=go.Data([trace_pw3,trace_pw5,trace_pw9,trace_pw11,trace_pw20,trace_max4,trace_min4])
layout=go.Layout(title="Piecewise Polynomial (2nd order)")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

# Plotting different methods results for "kc-pd-sql01" VM

In [49]:
[original_signal] = PL[(PL['Subject'] == 'kc-pd-sql01') & (PL['Window'] ==  20)].A.tolist()
x_axis = np.arange(1,len(original_signal)+1)
[MA_20W_signal] = MA[(MA['Subject'] == 'kc-pd-sql01') & (MA['Window'] ==  20)].F.tolist()
[WMA_20W_signal] = WMA[(WMA['Subject'] == 'kc-pd-sql01') & (WMA['Window'] ==  20)].F.tolist()
[ES_A8_signal] = ES[(ES['Subject'] == 'kc-pd-sql01') & (ES['Alpha'] ==  0.8)].F.tolist()
ES_A8_signal = ES_A8_signal[20:]
[EST_A8_signal] = EST[(EST['Subject'] == 'kc-pd-sql01') & (EST['Alpha'] ==  0.8)].F.tolist()
EST_A8_signal = EST_A8_signal[20:]
[WEST_3W_A8_signal] = WEST[(WEST['Subject'] == 'kc-pd-sql01') & (WEST['Alpha'] ==  0.8) & (WEST['Window'] ==  3)].F.tolist()
WEST_3W_A8_signal = WEST_3W_A8_signal[17:]
[PL_20W_signal] = PL[(PL['Subject'] == 'kc-pd-sql01') & (PL['Window'] ==  3)].F.tolist()
PL_20W_signal = PL_20W_signal[17:]
[PL2_20W_signal] = PL2[(PL2['Subject'] == 'kc-pd-sql01') & (PL2['Window'] ==  11)].F.tolist()
PL2_20W_signal = PL2_20W_signal[9:]
#trace_orignal = go.Scatter(x = range(1,len()))

In [50]:
len(PL2_20W_signal)

272

In [51]:
# [3, 5, 9, 11, 20]
trace_o = go.Scatter( x = x_axis,y=original_signal, marker={'size': "15"}, 
                    mode="lines", name='Original Signal')
trace_ma = go.Scatter( x = x_axis,y=MA_20W_signal, marker={'size': "5"}, 
                    mode="lines+markers", name='Moving Average')
trace_wma = go.Scatter( x = x_axis,y=WMA_20W_signal, marker={'size': "10"}, 
                    mode="markers", name='Weighted Moving Average')
trace_es = go.Scatter( x = x_axis,y=ES_A8_signal, marker={'size': "10"}, 
                    mode="lines+markers", name='Exponential Smoothening')
trace_est = go.Scatter( x = x_axis,y=EST_A8_signal, marker={'size': "15"}, 
                    mode="lines+markers", name='Exponential Smoothening w/ Trend')
trace_west = go.Scatter( x = x_axis,y=WEST_3W_A8_signal, marker={'size': "5"}, 
                    mode="markers", name='Moving Window Exponential Smoothening w/ Trend')
trace_pl = go.Scatter( x = x_axis,y=PL_20W_signal, marker={'size': "10"}, 
                    mode="lines+markers", name='Piecewise Linear')
trace_pl2 = go.Scatter( x = x_axis,y=PL2_20W_signal, marker={'size': "5"}, 
                    mode="markers", name='Piecewise 2nd order polynomial')


In [43]:

data=go.Data([trace_o,trace_ma,trace_wma])
layout=go.Layout(title="Original vs MA vs WMA")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

In [44]:
data=go.Data([trace_o,trace_es,trace_est,trace_west])
layout=go.Layout(title="Original vs ES vs EST vs WEST")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)

In [52]:
data=go.Data([trace_o,trace_pl,trace_pl2])
layout=go.Layout(title="Original vs Piecewise Linear vs Piecewise 2nd order polynomial")
figure=go.Figure(data=data,layout=layout)
py.iplot(figure)